In [3]:
# Bayesian vs XGBoost on the Same Dataset
# Dataset: UCI Bike Sharing (demand forecasting)
# Business problem: demand forecasting under uncertainty

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error
from sklearn.preprocessing import StandardScaler

import xgboost as xgb

import pymc as pm
import arviz as az

In [6]:
# Load data 

url = "bike_sharing_daily.csv" #"https://raw.githubusercontent.com/jbrownlee/Datasets/master/bike_sharing_daily.csv"
df = pd.read_csv(url)

# Target: daily bike rentals
y = df['cnt'].values

# Features (simplified)
X = df[[
    'temp',      # normalized temperature
    'hum',       # humidity
    'windspeed', # wind speed
    'workingday'
]].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [7]:
# XGBoost baseline 

xgb_model = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

xgb_mae = mean_absolute_error(y_test, xgb_preds)
xgb_rmse = root_mean_squared_error(y_test, xgb_preds)

print("XGBoost MAE:", xgb_mae)
print("XGBoost RMSE:", xgb_rmse)


XGBoost MAE: 1092.702392578125
XGBoost RMSE: 1332.0433349609375


In [8]:
# Bayesian hierarchical linear model 

# Standardize features for geometry
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

with pm.Model() as bayes_model:
    # Priors
    alpha = pm.Normal("alpha", 0, 10)
    beta = pm.Normal("beta", 0, 5, shape=X_train_s.shape[1])
    sigma = pm.HalfNormal("sigma", 10)

    # Linear model
    mu = alpha + pm.math.dot(X_train_s, beta)

    # Likelihood
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y_train)

    trace = pm.sample(
        1000,
        tune=1000,
        target_accept=0.9,
        chains=4,
        random_seed=42
    )


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, sigma]


/Users/gokhancetinkaya/miniconda3/envs/gokhancetinkaya.ai/lib/python3.11/site-packages/rich/live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/gokhancetinkaya/miniconda3/envs/gokhancetinkaya.ai/lib/python3.11/site-packages/pymc/step_methods/hmc/quadpotential.py:316: RuntimeWarning: overflow encountered in dot
  return 0.5 * np.dot(x, v_out)
/Users/gokhancetinkaya/miniconda3/envs/gokhancetinkaya.ai/lib/python3.11/site-packages/pymc/step_methods/hmc/quadpotential.py:316: RuntimeWarning: overflow encountered in dot
  return 0.5 * np.dot(x, v_out)


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1 seconds.


In [11]:
# Posterior predictive 

with bayes_model:
    posterior_pred = pm.sample_posterior_predictive(
        trace,
        var_names=["y_obs"],
        random_seed=42
    )

y_pp = posterior_pred.posterior_predictive["y_obs"].values
bayes_preds = y_pp.mean(axis=(0, 1))

bayes_mae = mean_absolute_error(y_train, bayes_preds)
bayes_rmse = root_mean_squared_error(y_train, bayes_preds)

print("Bayesian (train) MAE:", bayes_mae)
print("Bayesian (train) RMSE:", bayes_rmse)


Sampling: [y_obs]


/Users/gokhancetinkaya/miniconda3/envs/gokhancetinkaya.ai/lib/python3.11/site-packages/rich/live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Bayesian (train) MAE: 4356.666555202006
Bayesian (train) RMSE: 4757.040870023143


In [18]:
# Uncertainty intervals 
intervals = np.percentile(posterior_pred.posterior_predictive['y_obs'].values, [5, 95], axis=(0, 1))


In [19]:
intervals

array([[-1537.99634425, -1524.64407291, -1530.80875339, ...,
        -1484.73680763, -1493.31628028, -1522.92869787],
       [ 1905.75116454,  1873.61649922,  1929.27139453, ...,
         1974.05844775,  1912.73015667,  1994.95204516]], shape=(2, 548))

In [21]:
i = 0
bayes_preds[i], intervals[0, i], intervals[1, i]

(np.float64(226.20823405730818),
 np.float64(-1537.996344246879),
 np.float64(1905.751164541095))

In [22]:
print("Example prediction interval (first 5 samples):")
for i in range(5):
    print(f"Mean={bayes_preds[i]:.1f}, 5%={intervals[0, i]:.1f}, 95%={intervals[1, i]:.1f}")

# --- 7. Diagnostics ---
az.summary(trace, var_names=["alpha", "beta", "sigma"])


Example prediction interval (first 5 samples):
Mean=226.2, 5%=-1538.0, 95%=1905.8
Mean=184.7, 5%=-1524.6, 95%=1873.6
Mean=222.1, 5%=-1530.8, 95%=1929.3
Mean=244.3, 5%=-1554.5, 95%=1966.1
Mean=220.1, 5%=-1503.0, 95%=1991.4


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,219.803,10.112,200.314,238.024,0.146,0.165,4775.0,3281.0,1.0
beta[0],14.888,4.893,5.520,23.837,0.071,0.082,4740.0,2661.0,1.0
beta[1],-2.188,4.835,-12.070,6.079,0.069,0.080,4825.0,2977.0,1.0
beta[2],-5.533,4.912,-14.349,4.280,0.071,0.083,4810.0,2946.0,1.0
beta[3],1.343,4.980,-7.716,10.982,0.070,0.074,5056.0,3219.0,1.0
sigma,1042.292,5.052,1033.244,1052.022,0.069,0.076,5303.0,3176.0,1.0


In [25]:
bayes_model.y_obs

y_obs ~ Normal(f(alpha, beta), sigma)